# Julia 機器學習：LIBSVM 支撐向量機

## 作業 029：

請使用 SVM 來建立一個預測模型，來對手寫數字辨識資料集，建立一個分類模型，預測數字為何。

本作業需要使用到的套件有 MLDatasets，請在執行以下範例前先安裝。

```
] add MLDatasets
```

In [6]:
using Pkg
Pkg.add("MLDatasets")

  Updating registry at `C:\Users\TOM\.julia\registries\General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
 Resolving package versions...
 Installed CMakeWrapper ──── v0.2.3
 Installed URIParser ─────── v0.4.1
 Installed Blosc ─────────── v0.6.0
 Installed GZip ──────────── v0.5.1
 Installed BinDeps ───────── v1.0.1
 Installed DataDeps ──────── v0.7.2
 Installed HDF5 ──────────── v0.13.1
 Installed MLDatasets ────── v0.5.2
 Installed CMake ─────────── v1.2.0
 Installed HDF5_jll ──────── v1.10.5+5
 Installed BufferedStreams ─ v1.0.0
 Installed MAT ───────────── v0.8.0
  Updating `C:\Users\TOM\.julia\environments\v1.3\Project.toml`
  [eb30cadb] + MLDatasets v0.5.2
  Updating `C:\Users\TOM\.julia\environments\v1.3\Manifest.toml`
  [9e28174c] + BinDeps v1.0.1
  [a74b3585] + Blosc v0.6.0
  [e1450e63] + BufferedStreams v1.0.0
  [631607c0] + CMake v1.2.0
  [d5fb7624] + CMakeWrapper v0.2.3
  [124859b0] + DataDeps v0.7.2
  [92fee26a] + GZip v0.5.1
  [f67ccb44] + HDF5 v

In [7]:
using LIBSVM, MLDatasets, Statistics

┌ Info: Precompiling MLDatasets [eb30cadb-4394-5ae3-aed4-317e484a6458]
└ @ Base loading.jl:1273


## 讀取資料

呼叫 MNIST 資料集的過程中，會先去檢查以前是否有下載過，如果是第一次下載，過程中會詢問是否下載資料集，請回答 `y`。

In [8]:
train_X, train_y = MNIST.traindata(1:10000)
test_X,  test_y  = MNIST.testdata(1:1000)

This program has requested access to the data dependency MNIST.
which is not currently installed. It can be installed automatically, and you will not see this message again.

Dataset: THE MNIST DATABASE of handwritten digits
Authors: Yann LeCun, Corinna Cortes, Christopher J.C. Burges
Website: http://yann.lecun.com/exdb/mnist/

[LeCun et al., 1998a]
    Y. LeCun, L. Bottou, Y. Bengio, and P. Haffner.
    "Gradient-based learning applied to document recognition."
    Proceedings of the IEEE, 86(11):2278-2324, November 1998

The files are available for download at the offical
website linked above. Note that using the data
responsibly and respecting copyright remains your
responsibility. The authors of MNIST aren't really
explicit about any terms of use, so please read the
website to make sure you want to download the
dataset.



Do you want to download the dataset from ["http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz", "http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyt

(FixedPointNumbers.Normed{UInt8,8}[0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8; 0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8; … ; 0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8; 0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8]

FixedPointNumbers.Normed{UInt8,8}[0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8; 0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8; … ; 0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8; 0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8]

FixedPointNumbers.Normed{UInt8,8}[0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8; 0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8; … ; 0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8; 0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8]

...

FixedPointNumbers.Normed{UInt8,8}[0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8; 0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8; … ; 0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8; 0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8]

FixedPointNumbers.Normed{UInt8,8}[0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8; 0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8; … ; 0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8; 0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8]

FixedPointNumbers.Normed{UInt8,8}[0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8; 0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N

In [10]:
typeof(train_y)

Array{Int64,1}

## 資料轉換

In [11]:
# X = Matrix(iris[!, 1:4])
# y = Vector{String}(iris[!, :Species]);

train_X = Matrix(train_X[])
test_X = Matrix(test_X[])

MethodError: MethodError: no method matching Array{T,2} where T(::Base.ReinterpretArray{FixedPointNumbers.Normed{UInt8,8},3,UInt8,Array{UInt8,3}})
Closest candidates are:
  Array{T,2} where T(::AbstractArray{S,N}) where {S, N} at boot.jl:426
  Array{T,2} where T(!Matched::LinearAlgebra.SymTridiagonal{T,V} where V<:AbstractArray{T,1}) where T at D:\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.3\LinearAlgebra\src\tridiag.jl:110
  Array{T,2} where T(!Matched::LinearAlgebra.Tridiagonal{T,V} where V<:AbstractArray{T,1}) where T at D:\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.3\LinearAlgebra\src\tridiag.jl:506
  ...

In [ ]:
train_y = # convert to string type
test_y = # convert to string type

## 支撐向量機模型

In [ ]:
model = LIBSVM.fit!(SVC(), X, y)

## 預測

In [ ]:
ŷ = String[]
for i = 1:size(test_X, 1)
    y_ = LIBSVM.predict(model, Matrix(test_X[i, :]'))
    append!(ŷ, y_)
end

## 評估

In [ ]:
accuracy() = mean((predicted_labels .== y))*100

In [ ]:
accuracy()